In [7]:
from keras.layers import Dense, Activation, Embedding
import tensorflow as tf
from keras.layers import Input, Flatten, dot, concatenate, Dropout
from keras.layers import Layer
from tensorflow.python.keras import backend as K
monthInput = Input(shape=[1], dtype=tf.int32)
monthEmbedding = Embedding(12, 64)

In [8]:
class SurroundingSlots(Layer):
    def __init__(self, window_length, max_range, trainable=True, name=None, **kwargs):
        super(SurroundingSlots, self).__init__(name=name, trainable=trainable, **kwargs)
        self.window_length = window_length
        self.max_range = max_range

    # def build(self, inshape):
    #     1

    def call(self, x):
        surr = K.cast(x, dtype=tf.int32) + K.arange(start=-self.window_length, stop=self.window_length + 1, step=1)
        surrUnderflow = K.cast(surr < 0, dtype=tf.int32)
        surrOverflow = K.cast(surr > self.max_range - 1, dtype=tf.int32)

        return surr * (-(surrUnderflow + surrOverflow) + 1) + surrUnderflow * (surr + self.max_range) + surrOverflow * (surr - self.max_range)

    def compute_output_shape(self, inshape):
        return (inshape[0], self.window_length * 2 + 1)

In [11]:
class meanLayer(Layer):
    def __init__(self, trainable=True, name=None, **kwargs):
        super(meanLayer, self).__init__(name=name, trainable=trainable, **kwargs)

    def build(self, inshape):
        1

    def call(self, x):
        return K.mean(x, axis=1, keepdims=True)
        
    def compute_output_shape(self, inshape):
        return (inshape[0], 1, inshape[2])

In [10]:
surr=monthEmbedding(SurroundingSlots(window_length=3, max_range=6)(monthInput[2]))

In [13]:
type(monthInput)

keras.engine.keras_tensor.KerasTensor

: 

In [12]:
meanLayer()(surr)

<KerasTensor: shape=(7, 1) dtype=float32 (created by layer 'mean_layer')>